In [1]:
import pandas as pd
import numpy as np

from boruta import BorutaPy

In [50]:
fm = pd.read_csv('data/ft_2000_important.csv').replace({-np.inf: np.nan, np.inf:np.nan})
fm.shape

test_base = pd.read_csv('data/test.csv')[['Id', 'idhogar']]
test_base.shape

train = fm[fm['Target'].notnull()].copy()
test = fm[fm['Target'].isnull()].copy()

train_labels = np.array(train.pop('Target')).reshape((-1,))
train_ids = train.pop('idhogar')
test = test.drop(columns = 'Target')
test_ids = test.pop('idhogar')

for col in train:
    if train[col].dtype == 'object':
        train[col] = train[col].astype(np.float32)
        test[col] = test[col].astype(np.float32)
        
feature_names = list(train.columns)
train = np.array(train, dtype = 'float')
test = np.array(test, dtype = 'float')

from sklearn.preprocessing import Imputer

imputer = Imputer(strategy = 'median')
train = imputer.fit_transform(train.astype(np.float32))
test = imputer.transform(test.astype(np.float32))

train = np.nan_to_num(train)
test = np.nan_to_num(test)

train_labels = np.array(train_labels, dtype = 'float')

C:\Users\willk\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (572) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
np.all(np.isfinite(train))

True

In [52]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs = -1, max_depth = 5)
selector = BorutaPy(rf, n_estimators = 'auto', random_state = 50)

In [69]:
train_df = pd.DataFrame(train, columns = feature_names)
test_df = pd.DataFrame(test, columns = feature_names)

In [53]:
import warnings
warnings.filterwarnings('ignore' ,category = RuntimeWarning)
selector.fit(train, train_labels)

C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\w

C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\w

BorutaPy(alpha=0.05,
     estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=480, n_jobs=-1,
            oob_score=False,
            random_state=<mtrand.RandomState object at 0x00000243BE053480>,
            verbose=0, warm_start=False),
     max_iter=100, n_estimators='auto', perc=100,
     random_state=<mtrand.RandomState object at 0x00000243BE053480>,
     two_step=True, verbose=0)

In [54]:
selector.n_features_

270

In [78]:
features_to_keep = []
for i in list(np.where(selector.ranking_ == 1)[0]):
    features_to_keep.append(feature_names[i])

In [79]:
train_r = selector.transform(train)
test_r = selector.transform(test)

In [80]:
def macro_f1_score(labels, predictions):
    # Reshape the predictions as needed
    predictions = predictions.reshape(len(np.unique(labels)), -1 ).argmax(axis = 0)
    
    metric_value = f1_score(labels, predictions, average = 'macro')
    
    # Return is name, value, is_higher_better
    return 'macro_f1', metric_value, True

def model_gbm(hyperparameters, features, labels, 
              test_features, test_ids, submission_base, 
              nfolds = 5, return_preds = False):
    """Model using the GBM and cross validation.
       Trains with early stopping on each fold.
       Hyperparameters probably need to be tuned."""
    
    feature_names = list(features.columns)
    
    # Model with hyperparameters selected from previous work
#     model = lgb.LGBMClassifier(boosting_type = 'gbdt', n_estimators = 10000, max_depth = -1,
#                                learning_rate = 0.025, metric = 'None', min_child_samples = 30,
#                                reg_alpha = 0.35, reg_lambda = 0.6, num_leaves = 15, 
#                                colsample_bytree = 0.85, objective = 'multiclass', 
#                                class_weight = 'balanced', 
#                                n_jobs = -1)

    model = lgb.LGBMClassifier(**hyperparameters, 
                               objective = 'multiclass', n_jobs = -1, 
                               n_estimators = 10000, metric = 'None')
    
    # Using stratified kfold cross validation
    strkfold = StratifiedKFold(n_splits = nfolds, shuffle = True)
    predictions = pd.DataFrame()
    importances = np.zeros(len(feature_names))
    
    # Convert to arrays for indexing
    features = np.array(features)
    test_features = np.array(test_features)
    labels = np.array(labels).reshape((-1 ))
    
    valid_scores = []
    
    # Iterate through the folds
    for i, (train_indices, valid_indices) in enumerate(strkfold.split(features, labels)):
        # Dataframe for 
        fold_predictions = pd.DataFrame()
        
        # Training and validation data
        X_train = features[train_indices]
        X_valid = features[valid_indices]
        y_train = labels[train_indices]
        y_valid = labels[valid_indices]
        
        # Train with early stopping
        model.fit(X_train, y_train, early_stopping_rounds = 100, 
                  eval_metric = macro_f1_score,
                  eval_set = [(X_train, y_train), (X_valid, y_valid)],
                  eval_names = ['train', 'valid'],
                  verbose = 200)
        
        # Record the validation fold score
        valid_scores.append(model.best_score_['valid']['macro_f1'])
        
        # Make predictions from the fold
        fold_probabilitites = model.predict_proba(test_features)
        
        # Record each prediction for each class as a column
        for j in range(4):
            fold_predictions[(j + 1)] = fold_probabilitites[:, j]
            
        fold_predictions['idhogar'] = test_ids
        fold_predictions['fold'] = (i+1)
        predictions = predictions.append(fold_predictions)
        
        importances += model.feature_importances_ / nfolds   
        
        display(f'Fold {i + 1}, Validation Score: {round(valid_scores[i], 5)}, Estimators Trained: {model.best_iteration_}')

    feature_importances = pd.DataFrame({'feature': feature_names,
                                        'importance': importances})
    valid_scores = np.array(valid_scores)
    display(f'{nfolds} cross validation score: {round(valid_scores.mean(), 5)} with std: {round(valid_scores.std(), 5)}.')
    
    # If we want to examine predictions don't average over folds
    if return_preds:
        predictions['Target'] = predictions[[1, 2, 3, 4]].idxmax(axis = 1)
        predictions['confidence'] = predictions[[1, 2, 3, 4]].max(axis = 1)
        return predictions, feature_importances
    
    # Average the predictions over folds
    predictions = predictions.groupby('idhogar', as_index = False).mean()
    
    # Find the class and associated probability
    predictions['Target'] = predictions[[1, 2, 3, 4]].idxmax(axis = 1)
    predictions['confidence'] = predictions[[1, 2, 3, 4]].max(axis = 1)
    predictions = predictions.drop(columns = ['fold'])
    
    # Merge with the base to have one prediction for each individual
    submission = submission_base.merge(predictions[['idhogar', 'Target']], on = 'idhogar', how = 'left').drop(columns = ['idhogar'])
        
    submission['Target'] = submission['Target'].fillna(4).astype(np.int8)
    
    # return the submission and feature importances
    return submission, feature_importances, valid_scores

In [81]:
hyp_OPTaaS = { 'boosting_type': 'dart',
              'colsample_bytree': 0.9843467236959204,
              'learning_rate': 0.11598629586769524,
              'min_child_samples': 44,
              'num_leaves': 49,
              'reg_alpha': 0.35397370408131534,
              'reg_lambda': 0.5904910774606467,
              'subsample': 0.6299872254632797,
              'subsample_for_bin': 60611}

In [61]:
selector.ranking_

array([1476, 1437,    1, ..., 1618,  342,  952])

In [62]:
ranks = selector.ranking_
len(ranks)

2014

In [64]:
len(feature_names)

2014

In [83]:
train_r_df = pd.DataFrame(train_r, columns = features_to_keep)
test_r_df = pd.DataFrame(test_r, columns = features_to_keep)

In [84]:
feature_rankings = pd.DataFrame({'rank': list(ranks), 'feature': feature_names})
feature_rankings.sort_values('rank', inplace = True)
feature_rankings.head(10)

,rank,feature
1219,1,rooms / PERCENT_TRUE(ind.instlevel2)
308,1,dependency / rooms
1818,1,r4t1 / SUM(ind.escolari)
864,1,STD(ind.escolari) / PERCENT_TRUE(ind.estadociv...
303,1,r4m3 / hogar_adul
300,1,meaneduc / bedrooms
298,1,hogar_nin / qmobilephone
875,1,dependency / SUM(ind.escolari)
888,1,PERCENT_TRUE(ind.female) / meaneduc
862,1,hogar_nin / MEAN(ind.escolari)


In [85]:
from collections import Counter
Counter(ranks)[1]

270

In [87]:
import pandas as pd
import numpy as np

import lightgbm as lgb

# Evaluation of the model
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score

In [88]:
submission, feature_importances, valid_scores = model_gbm(hyp_OPTaaS, train_r_df, train_labels,
                                                          test_r_df, test_ids, test_base)

Training until validation scores don't improve for 100 rounds.
[200]	train's macro_f1: 0.904415	valid's macro_f1: 0.364316
Early stopping, best iteration is:
[113]	train's macro_f1: 0.790664	valid's macro_f1: 0.409807


'Fold 1, Validation Score: 0.40981, Estimators Trained: 113'

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[82]	train's macro_f1: 0.778983	valid's macro_f1: 0.436033


'Fold 2, Validation Score: 0.43603, Estimators Trained: 82'

Training until validation scores don't improve for 100 rounds.
[200]	train's macro_f1: 0.911415	valid's macro_f1: 0.385574
Early stopping, best iteration is:
[114]	train's macro_f1: 0.807878	valid's macro_f1: 0.408978


...

Training until validation scores don't improve for 100 rounds.
[200]	train's macro_f1: 0.907618	valid's macro_f1: 0.364881
Early stopping, best iteration is:
[120]	train's macro_f1: 0.816308	valid's macro_f1: 0.394096


'Fold 4, Validation Score: 0.3941, Estimators Trained: 120'

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[24]	train's macro_f1: 0.581404	valid's macro_f1: 0.38216


'Fold 5, Validation Score: 0.38216, Estimators Trained: 24'

'5 cross validation score: 0.40621 with std: 0.01808.'

In [89]:
feature_importances.sort_values('importance', ascending = False).head()

,feature,importance
132,rooms / MAX(ind.age),132.4
115,MIN(ind.age / escolari),114.4
212,r4h2 / MEAN(ind.age),94.6
220,STD(ind.age) / rooms,88.0
260,qmobilephone / STD(ind.age),87.4


In [90]:
from sklearn.feature_selection import SelectFdr

In [91]:
select_f = SelectFdr()

In [92]:
select_f.fit(train, train_labels)

C:\Users\willk\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\willk\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:368: RuntimeWarning: overflow encountered in square
  X = X ** 2
C:\Users\willk\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:99: RuntimeWarning: overflow encountered in square
  square_of_sums_args = [s ** 2 for s in sums_args]
C:\Users\willk\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:100: RuntimeWarning: invalid value encountered in subtract
  sstot = ss_alldata - square_of_sums_alldata / float(n_samples)
C:\Users\willk\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:104: RuntimeWarning: invalid value encountered in subtract
  ssbn -= square_of_sums_alldata / float(n_samples)


SelectFdr(alpha=0.05, score_func=<function f_classif at 0x00000243A9736598>)

In [93]:
select_f.pvalues_

array([2.4080304e-13, 6.1064126e-10, 1.4012985e-45, ..., 7.6102368e-03,
       2.1672032e-29, 4.1048896e-02], dtype=float32)

In [32]:
np.all(np.isfinite(train_array))

False